In [1]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import glob
import os
import cv2

/Users/muhammadsaad/Downloads/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Directory containing the images
directory = './img/'
output_directory = './student_img'

# List of image file extensions to read
image_exts = ['jpg', 'jpeg', 'png', 'bmp']

for ext in image_exts:
    for filename in glob.glob(os.path.join(directory, f'*.{ext}')):
        image_name = os.path.splitext(os.path.basename(filename))[0]
        
        # Remove spaces and brackets from the image name
        image_name = image_name.replace(' ', '').replace('(', '').replace(')', '')
        
        # Read the image
        img = cv2.imread(filename)
        
        # Create the output directory for the image
        image_folder = os.path.join(output_directory, image_name)
        os.makedirs(image_folder, exist_ok=True)
        
        # Save the image in the corresponding folder
        output_path = os.path.join(image_folder, os.path.basename(filename))
        cv2.imwrite(output_path, img)
#         print(output_path)


In [3]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('student_img') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


### Saving data into data.pt file

In [4]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

### Matching face id of the given photo with available data from data.pt file

In [8]:

def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))


result = face_match('student_img/pic4/pic4.png', 'data.pt')

print('Face matched with: ',result[0], 'With distance: ',result[1])

Face matched with:  pic4 With distance:  0.0
